# Efficiently Selecting and Processing Raster and Vector Data

This workflow introduces how to search for, acquire and process raster data efficiently. Specfically, we demonstrate finding [Sentinel-2](https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-2) satellite imagery relevant for an area and time, in this case, a time range of a few months in 2022 over a subdistrict in India.

Note: some portions of this notebook are inspired by the [Introduction to Geospatial Raster and Vector Data with Python course](https://carpentries-incubator.github.io/geospatial-python/index.html).


## Main Objectives

- **Access Sentinel-2 data**: Locate and retrieve relevant Sentinel-2 data for a specific area and timeframe via STAC.
- **Inspect and visualize raster data**: Examine metadata, including projections, bands, dimensions, no data pixels. Plot raster data correctly.
- **Process multi-spectral raster data**: Work with vector data to establish bounds, manage coordinate systems, and set up the area of interest (AOI).
- **Interpret time-series raster data**: Learn to explore the time dimension for raster data and search for temporal patterns.
- **Cloud and nodata masking**: Handle clouds and no data pixels.
- **Compositing**: Create composites from multiple image scenes.
- **Speed up raster processing with Dask**: Learn how to optimize raster processing steps using a parallel processing library.
- **Raster band math**: Calculate relevant spectral indices using Sentinel-2 data.
- **Scalably visualize vector data with Lonboard**: Load a field boundaries with 10000 polygons and visualize it with Lonboard.
- **Calculate zonal statistics with the field boundaries**: Show how to calculate zonal statistics for a spectral index.


At the end, you'll be able to efficiently process raster imagery for a region and time of interest!

Let's begin by importing our necessary packages.

In [ ]:
import dask
import geopandas as gpd
import odc.stac
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pystac_client
import rasterio
import requests
import rioxarray
import time
import xarray as xr
from geopandas import GeoSeries, GeoDataFrame
from lonboard import Map, PolygonLayer, viz
from pyproj import CRS
from rasterstats import zonal_stats
from shapely.geometry import box, Point, shape
from tqdm.auto import tqdm

## Accessing raster data

We'll select an area of interest by randomly sampling a subdistrict within India. This is what we will use to obtain geographic bounds required for selecting and acquiring raster data. 

Note that we set the coordinate reference system (CRS) to the projection that is used to load web map tiles and store coordinate metadata in the STAC catalog, `EPSG:4326 (WGS 84)`. 

In [ ]:
# Read in the subdistricts boundaries dataset
subdistricts = gpd.read_file("./KA_subdistrict/KA_subdistrict.shp")
# Shuffle the dataset items
subdistricts.sample(frac=1)
#subdistrict = subdistricts[subdistricts["subdistric"] == "Bangalore-South"]  # UID = 97
# Select the first subdistrict
subdistrict = subdistricts.head(1).to_crs("EPSG:4326")
subdistrict

Obtain a bounding box for the sampled subdistrict. This is for the image search query we will execute shortly.

In [ ]:
bbox_coords = subdistrict.total_bounds 
bounding_box = box(*bbox_coords)

The area of interest can be seen in the following:

In [ ]:
viz(subdistrict)

Let’s dive into how to search for and retrieve raster data.

The [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/en) is an open standard created to standardize the organization and description of geospatial assets. It has come to serve a crucial role in the geospatial industry by providing a consistent framework and terminology for indexing and accessing Earth observation data along with relevant metadata. This specification allows for streamlined searching, browsing, and retrieval of geospatial information.

You can explore geospatial datasets using tools like the [STAC browser](https://radiantearth.github.io/stac-browser/#/?.language=en), which provides an up-to-date interface for navigating STAC catalogs. For instance, the ["Earth Search" catalog](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/?.language=en), which includes Sentinel-2 imagery [hosted on AWS](https://registry.opendata.aws/sentinel-2-l2a-cogs/), is a great starting point. 

To retrieve data from a catalog, locate its API URL in the STAC Browser by clicking the "Source" button in the top-right corner. This URL serves as a gateway to access and interact with the catalog's data. For the Earth Search catalog, the API URL would be displayed in this section, enabling you to programmatically access the archive.

In [ ]:
# Access AWS STAC for Sentinel-2 Data
aws_stac_url = "https://earth-search.aws.element84.com/v1"

# You can query a STAC API endpoint from Python using the pystac_client library:
stac_client = pystac_client.Client.open(aws_stac_url)

In the upcoming steps, we will retrieve imagery from the Sentinel-2 L2A collection, which consists of data products processed to Level 2A. This processing level provides bottom-of-atmosphere (BOA) reflectance, making the data ready for analysis without additional atmospheric correction. 

The data is stored in the **Cloud Optimized GeoTIFF (COG)** format, a type of GeoTIFF specifically designed for efficient access in cloud environments. Unlike traditional GeoTIFFs, COGs are structured with internal data blocks that enable partial retrieval via HTTP requests, eliminating the need to download the entire file. Furthermore, they often include **overviews**, which are lower-resolution versions of the image. These overviews allow users to quickly access less detailed images when high resolution isn’t required, optimizing data transfer and reducing load times for online applications. For more about COGs, visit the [Cloud Optimized GeoTIFF website](http://cogeo.org/).

We parameterize our search for available Sentinel-2 scenes in the `sentinel-2-l2a` collection that meet the following criteria:
- intersect with a provided bounding box (boundaries of the area of interest);
- were captured between January 1, 2022, and December 31, 2022;
- have less than 5% cloud cover;
- have less than 5% invalid pixels.

In [ ]:
s2_search = stac_client.search(
    collections=["sentinel-2-l2a"], # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)
    bbox=list(bbox_coords),
    datetime="2022-01-01/2022-12-31",
    query={"eo:cloud_cover": {"lt": 5}, "s2:nodata_pixel_percentage": {"lt": 5}}, 
)

At this point, only metadata has been retrieved, meaning the actual image data has not yet been loaded into memory. However, it’s important to note that metadata alone can occupy significant memory, especially when the search criteria match a large number of images. To handle this efficiently, you can limit the number of search results by using the parameter `limit=n`, where `n` specifies the maximum number of items to return.

In [ ]:
# Retrieve all items (still just metadata) from search results
s2_items = s2_search.item_collection()

After running the query to identify image scenes that match our search criteria, we can check the total number of matches using the `.matched()` method. Keep in mind that this number may change over time as new data continues to be added to the catalog.

In [ ]:
print(s2_search.matched())

Which should equate to the number of items collected:

In [ ]:
len(s2_items)

Each item corresponds to a single image scene, with metadata that includes details such as the scene's spatial geometry and the time it was captured. These details are accessible through the item's properties.


We can see an example for the first item in the search results:

In [ ]:
item = s2_items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

As noted earlier, only the metadata has been retrieved so far. Next, we will access the raster data, which consists of the image pixels for the selected scenes. In STAC terminology, these image files are referred to as "assets," with each band of an image represented by a separate asset. A straightforward way to load this image data is by using the URLs provided in the assets attribute of the corresponding item.

In [ ]:
assets = s2_items[0].assets  # first item's asset dictionary
print(assets.keys())

We can see a description of the available assets for the respective sensor/instrument like so:

In [ ]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

The Sentinel-2 Level-2A data product offers a variety of assets, including raster files for each optical band captured by the multispectral instrument, a true-color thumbnail image ("visual"), metadata about the instrument, and scene classification layers ("SCL"). Let's retrieve the URLs associated with these assets to access the respective data.








In [ ]:
print(assets["thumbnail"].href)

The URL points to the image's location in cloud storage. Since the raster data is stored remotely, we can utilize a library that enables direct access to the image without needing to download it to the local file system beforehand. This library is called `rioxarray`. 

In [ ]:
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)

Optionally, we can then save the raster data to a file on disk if needed:

In [ ]:
# save whole image to disk
#nir.rio.to_raster("nir.tif")

Working with large rasters, such as the 10-meter NIR band containing over 100 million pixels, can be computationally intensive. To optimize processing, it's often practical to focus on a smaller subset of the data. With the Cloud Optimized GeoTIFF (COG) format, we can efficiently download just the required section instead of the entire file. Before doing so, let's examine the internal tile size of this band in the COG.

In [ ]:
print(nir.shape)
tile_size = nir.rio._manager.acquire().block_shapes
tile_size

In this case, we specify that we want to download the first band in the TIFF file and extract a subset by slicing the width and height dimensions.

In [ ]:
# save portion of an image to disk
#nir[0,1024:2048,1024:2048].rio.to_raster("nir_subset.tif")

We can read the newly saved image subset and confirm the size is what we expect:

In [ ]:
#nir = rioxarray.open_rasterio("nir_subset.tif")

In [ ]:
#nir.shape

## Working with Raster Data

This section introduces key concepts, tools, and attributes related to raster data in Python. It also covers strategies for managing missing or invalid values in raster datasets.

We’ll use the `rioxarray` library as our main tool for raster processing. Built on `rasterio` (a library for working with raster files) and `xarray` (designed for multi-dimensional arrays), `rioxarray` enhances these tools by providing additional geospatial functionality. For example, it includes the `open_rasterio` function for reading raster files and extends `xarray` objects, such as `Dataset` and `DataArray`, with geospatial methods via the `rio` accessor. These features become available once you import `rioxarray`.

To demonstrate, we will use the `nir09` band from the first image scene retrieved. This can be loaded with the `rioxarray.open_rasterio()` function, using the URL (Hypertext Reference or `href`) of the band asset.

In [ ]:
raster_b9 = rioxarray.open_rasterio(s2_items[0].assets["nir09"].href)

We can quickly inspect the shape and attributes of the newly opened `nir09` dataset we titled `raster_b9` by printing the variable name. 

In [ ]:
raster_b9

When `rioxarray.open_rasterio()` is called, it retrieves the raster data from either local or remote storage and returns an `xarray.DataArray`. This object is assigned to a variable, such as `raster_b9`. While `xarray` itself also returns a `DataArray`, it doesn't include essential geospatial metadata like scene geometry or projection details. Without `rioxarray`, you can use `NumPy` functions or Python's math operations on a `DataArray` as you would with a `NumPy` array, but you won’t have access to the geospatial information.

The output of the `DataArray` includes the variable name, along with details such as the number of bands (1), width in pixels (1830), and height in pixels (1830), as well as the pixel data type (e.g., unsigned integer `uint16`).

Thanks to `rioxarray`, the `DataArray` also includes spatial coordinates (x and y) and band information, each with its own data type—`float64` for spatial coordinates and `int64` for bands. Additionally, `rioxarray` allows us to access important geospatial attributes, such as the coordinate reference system (CRS) using `.rio.crs` and the bounding box using `.rio.bounds()`. Note that while most metadata can be accessed directly via attributes like `.rio.crs`, some methods, like `.rio.bounds()`, require parentheses to retrieve the data.

In [ ]:
print(raster_b9.rio.crs)
print(raster_b9.rio.nodata)
print(raster_b9.rio.bounds())
print(raster_b9.rio.width)
print(raster_b9.rio.height)
print(raster_b9.rio._manager.acquire().block_shapes)

The Coordinate Reference System for `raster_b9.rio.crs` is returned as `EPSG:32643`. The no-data value is set to `0`, and the bounding box corners of our raster are returned by the output of `.bounds()`. The height and width in number of pixels are returned from `.rio.height` and `.rio.width`, respectively.

## Visualizing raster data

We've reviewed the attributes of our raster. Now let's see the raw values of the array with `.values`:

In [ ]:
raster_b9.values

Displaying this allows us to quickly examine the array values, typically by viewing the pixel values at the image's corners. Since the raster is loaded as a `DataArray` in Python, we can easily visualize it using `DataArray.plot()`, much like how we would plot a `pandas` `DataFrame`, with a single line of code.

In [ ]:
raster_b9.plot()

With `rioxarray`, plotting a raster becomes straightforward, with spatial coordinates automatically labeled along the x and y axes for clarity.

The visualization depicts pixel values for the spectral band `nir09` over the selected area of interest. According to Sentinel-2 documentation, this band, with a central wavelength of 945 nm, is particularly sensitive to water vapor. It has a spatial resolution of 60 meters, which, while lower than some other bands, is ideal for quick analyses and demonstrations.

It's worth noting that the `band=1` label in the image title refers to the order of bands within the `DataArray` and not specifically to the Sentinel-2 band asset `nir09`.

The image reveals high reflectance values in cloudy pixels, while other areas show lower contrast, consistent with the band’s sensitivity to water vapor. To enhance color contrast and better interpret the data, you can use the `robust=True` option, which restricts the display range to the 2nd and 98th percentiles of pixel values.

In [ ]:
raster_b9.plot(robust=True)

This functionality lets you adjust the color limits to better reflect the majority of pixel values in the image. 

If the `robust=True` option doesn't provide a satisfactory representation, you can further refine the visualization by manually setting the `vmin` and `vmax` parameters. In some cases, defining a value range may produce a more tailored plot.

In [ ]:
raster_b9.plot(vmin=100, vmax=7000)

Now, if we want to plot a subset of this image to focus more on a specific feature of interest, we can select a region of pixels again. Let's look at the internal tiling for this band of the COG:

In [ ]:
print(raster_b9.shape)
tile_size = raster_b9.rio._manager.acquire().block_shapes
tile_size

Notice that the tile size is smaller because the resolution for this band (`nir09`) is lower (60 meters) than that of the `nir` (10 meters). 

In [ ]:
# Calculate the center coordinates of the image
center_x, center_y = raster_b9.sizes["x"] // 2, raster_b9.sizes["y"] // 2

# Select a crop region using .isel()
raster_b9_subset = raster_b9.isel(
    x=slice(center_x - tile_size[0][0]*3, center_x + tile_size[0][0]),
    y=slice(center_y - tile_size[0][0]*2, center_y + tile_size[0][0]*2)
)

print(raster_b9_subset.shape)

raster_b9_subset.plot(robust=True)
plt.title("Crop of Sentinel-2 COG (NIR)")
plt.xlabel("X Pixel")
plt.ylabel("Y Pixel")
plt.show()

## Deciphering the Raster Coordinate Reference System (CRS)

An essential detail to examine in geospatial data is the Coordinate Reference System (CRS), which defines how spatial data is mapped to the Earth’s surface. Using the `.rio.crs` method, you can access the CRS information of your dataset. This method retrieves the CRS string associated with your `DataArray`.

In [ ]:
print(raster_b9.rio.crs)

EPSG codes are a standardized and concise way to identify specific Coordinate Reference Systems (CRS). To explore detailed information about a CRS, such as its unit of measurement, the `pyproj` library can be used. This library is designed to manage and perform operations related to CRS definitions.

In [ ]:
epsg = raster_b9.rio.crs.to_epsg()
crs = CRS(epsg)
crs

The ``CRS`` class from the ``pyproj`` library enables us to create a CRS object. We can retrieve specific information about a CRS from this, as well as obtain a basic summarization of the associated CRS.

One especially valuable attribute is ``area_of_use``, which indicates the geographic boundaries for which the CRS is designed to be applied.

In [ ]:
crs.area_of_use

In [ ]:
# More on the various attributes accessible for the CRS class can be viewed with the following:
# help(crs)

The `pyproj` CRS summary outlines all the details of a Coordinate Reference System (CRS) that Python or GIS software may utilize. For example, it describes the projection as **UTM Zone 43N**, part of the Universal Transverse Mercator (UTM) system, which divides the globe into 60 zones, each covering 6 degrees of longitude. 

This specific CRS uses the **WGS84 datum** as its reference point and applies a Cartesian coordinate system with two axes: `easting` and `northing`, measured in meters. Its geographic range spans from **72°E to 78°E** longitude and **0.0°N to 84.0°N** latitude in the northern hemisphere. 

The **Transverse Mercator** projection method is employed, optimal for regions with narrow longitudinal widths, such as UTM zones. Additionally, the CRS attributes include the mechanism for projecting coordinates onto a flat (Cartesian) plane. While the UTM system includes zones, not all CRSs are zone-specific, and this detail pertains specifically to UTM-based systems.

## Calculate Raster Statistics
Knowing basic statistical values of a raster dataset can be valuable. We can obtain some, such as minimum, maximum, mean, and standard deviation quite easily for `xarray.DataArray`s like so:

In [ ]:
print(raster_b9.min())
print(raster_b9.max())
print(raster_b9.mean())
print(raster_b9.std())

So with that, we can get key statistical values such as the minimum, maximum, mean, and standard deviation, along with the data type of the pixels. However, if you're interested in calculating specific quantiles, `xarray` provides the `.quantile()` method. For example, to determine the 25th and 75th percentiles, you can try the following:

In [ ]:
print(raster_b9.quantile([0.25, 0.75]))

## Dealing with Missing Data

So far, we've visualized a band from a Sentinel-2 image and calculated its statistics. However, it's critical to account for missing data, often indicated by a "nodata" value in raster datasets. This special value represents pixels where no valid data is available, typically due to sensor limitations or incomplete coverage. Missing data frequently appears along the raster's edges when the dataset doesn't fully cover the analyzed region.

Raster datasets inherently have a rectangular structure, so areas without data—such as boundaries not covered by the sensor—are assigned the nodata value. For example, in this dataset, the nodata value (`raster_b9.rio.nodata`) is **0**. When visualizing the data or calculating statistics, these missing values can be mistakenly treated as valid, leading to skewed results. For instance, including nodata pixels with a value of zero could lower the 25th percentile or other metrics inaccurately.

To handle this, missing data can be represented as `NaN` (Not a Number), which ensures it is excluded from calculations. This transformation can be achieved by setting the parameter `masked=True` when loading the raster dataset. This approach provides a clearer distinction between valid and missing data, preventing distortion in analyses or visualizations.

In [ ]:
raster_b9 = rioxarray.open_rasterio(s2_items[0].assets["nir09"].href, masked=True)

You can also utilize the `where` function to filter out all the pixels that differ from the raster's `nodata` value:

In [ ]:
raster_b9.where(raster_b9!=raster_b9.rio.nodata)

Both approaches will replace the nodata value of **0** with `NaN`, allowing missing data to be excluded from calculations. This ensures that recalculated statistics are accurate, as the nodata pixels are no longer treated as valid data points. 

By the way, a tip for a cleaner output: you can use the `.values` attribute when applying statistical functions. This returns only the computed values, without including any of the object metadata associated with the raster dataset. This streamlined result is particularly useful for further numerical analysis or visualization, as it provides just the values without the metadata.

In [ ]:
print(raster_b9.min().values)
print(raster_b9.max().values)
print(raster_b9.mean().values)
print(raster_b9.std().values)

It's worth mentioning that replacing `0` with `nan` to represent missing data will cause a change in the data type of the `DataArray` from integers to floats. This is an important consideration if the data type plays a crucial role in your specific use case or application.

## Incorporating multiple bands 

Up to now, we have worked with single-band rasters, focusing on the `nir` and later the `nir09` bands from a Sentinel-2 scene. To obtain a more easy-to-interpret RGB "true-color" depiction of the area, we can utilize the True Color Image (TCI) asset. This asset provides a high-resolution visualization by combining three optical bands—Red, Green, and Blue (RGB)—captured by the Sentinel-2 satellite’s MultiSpectral Instrument (MSI). With a spatial resolution of 10 meters, the TCI functions as a clear and detailed view of the area, making it ideal for quick visual assessments and analysis. Like the `nir09` band, we can load the TCI using:

In [ ]:
raster_overview = rioxarray.open_rasterio(s2_items[0].assets['visual'].href, overview_level=3)
raster_overview

We specified the argument `overview_level=3`, which corresponds to the spatial resolution determined by the level of downsampling in Sentinel-2 COGs. The native spatial resolution for Sentinel-2's True Color Image (TCI) bands is 10 meters. However, each overview level reduces this resolution by a factor of 2.

The resolution at different overview levels is as follows:
- Level 0 (native): 10 meters
- Level 1: 20 meters
- Level 2: 40 meters
- Level 3: 80 meters

When loading GeoTIFFs with the `open_rasterio()` function and checking the shape, the band dimension will appear first. For instance, in the `xarray.DataArray` object, the shape would be `(band: 3, y: 687, x: 687)`, showing three bands. It's always a good idea to inspect the shape of the raster array both after loading it and during any data manipulations (such as calculating band ratios or warping). This is especially important since many functions, particularly those for image plotting, expect the raster to have a specific shape (e.g., 1 or 3 channels arranged in a certain order). You can easily verify the shape using the `.shape` attribute.

In [ ]:
raster_overview.shape

You can visualize the multi-band data using the `DataArray.plot.imshow()` function:

In [ ]:
raster_overview.plot.imshow()

Keep in mind that the `DataArray.plot.imshow()` function is designed to work with image arrays that have three channels, which corresponds to the RGB color model. It is not suitable for arrays containing more than three channels. However, you can create a false-color image by using different channels in place of the typical RGB.

As shown in the previous figure, the true-color image may appear stretched. To correct the aspect ratio, you can adjust the settings in `DataArray.plot.imshow()`. Since the height-to-width ratio is 1:1 (verified by the `rio.height` and `rio.width` attributes), it's best to set the aspect ratio to 1. For example, you might choose to set the plot size to 5 inches and specify `aspect=1`. Keep in mind that when using the `aspect` argument, you must also define the plot size, as indicated in the `DataArray.plot.imshow()` documentation.

In [ ]:
raster_overview.plot.imshow(size=5, aspect=1)

Now, let's consider working not only with visual bands but also with other bands like the `nir` band. We'll introduce a new tool called [Open Data Cube (ODC)](https://www.opendatacube.org/), which we imported as `odc`, to help load multi-spectral time series raster data from STAC into an `xarray.Dataset`. The `odc` library provides an extension for handling raster data from STAC-compliant catalogs. We'll use this to more efficiently manage the Sentinel-2 data we previously queried.

In the following cell, the process is as follows:

1. **`s2_data = odc.stac.load(...)`**:
   - This loads the actual raster data from the Sentinel-2 items we retrieved earlier. The `load` function from `odc` fetches the data associated with the metadata.

2. **`items=s2_items`**:
   - Here, `s2_items` represents the Sentinel-2 scenes we collected from the STAC catalog, which contain the metadata and location information.

3. **`bands=["red", "green", "blue", "nir", "scl"]`**:
   - This specifies the spectral bands to load, including Red, Green, Blue, Near-Infrared (NIR), and the Scene Classification Layer (SCL), which helps identify cloudy pixels.

4. **`bbox=box_coords`**:
   - The `bbox` argument defines the **Area of Interest (AOI)** by specifying the geographic bounding box using coordinates in the format `[min_longitude, min_latitude, max_longitude, max_latitude]`, which corresponds to the region we are interested in.

5. **`progress=tqdm`**:
   - The `progress` argument integrates `tqdm`, a library that shows a progress bar, helping track the loading process, especially when dealing with large datasets.

**IMPORTANT:**
When the `bbox` argument is provided, `odc` will automatically clip the raster data to match the bounding box. This ensures that only the relevant data for the specified region is returned.

The resulting `s2_data` is an `xarray.Dataset` that we can use for further analysis.

In [ ]:
s2_data = odc.stac.load(
    items=s2_items[:12],
    bands=["red", "green", "blue", "nir", "scl"],
    bbox=bbox_coords,
    progress=tqdm,
    chunks={'x': 1024, 'y': 1024, 'time': -1},
)

The `odc.stac.load()` function returns an `xarray.Dataset` instead of an `xarray.DataArray` because we are loading multiple spectral bands (e.g., "red", "green", "blue", "nir", "scl"). An `xarray.Dataset` is used to store multiple variables, or bands, each represented as a `DataArray`. A `DataArray` typically holds a single variable (e.g. band) along with its associated dimensions and coordinates. Since we are loading more than one band, the function returns a `Dataset`, which is essentially a collection of `DataArrays`, each corresponding to one of the requested bands.

The `xarray.Dataset` structure allows you to work with related variables (e.g., spectral bands) that share common coordinates, such as latitude, longitude, and time. This structure is particularly useful when performing operations that involve multiple bands, like creating a composite RGB image or conducting multi-band analyses. 

If you were only loading a single band, the function would return a `DataArray`, which is more suited to handling one variable. However, since multiple bands are requested, a `Dataset` is the appropriate return type.

In [ ]:
s2_data

Notice that the time dimension is automatically interpreted and aligned with the number of items returned from `s2_search.matched()` and `len(s2_items)` earlier. This means that the dataset now contains multiple Sentinel-2 images, each corresponding to different timestamps and overlapping the area of interest (AOI). By default, `odc` stacks these images in a time series, without assuming any manipulation is required on the time dimension.

To visualize the dataset, we can composite the images over time, reducing the temporal dimension to a single layer where each pixel represents the average of all available timestamps. This approach gives us a temporal depth of 1, providing a composite that reflects changes over time, simplifying the data for easier interpretation.

In [ ]:
s2_data_composite = s2_data.mean(dim='time')

Notice the new shape of the dataset:

In [ ]:
s2_data_composite

Also notice here that the aspect ratio is not perfectly 1:1, so let's compute the correct aspect ratio to use when plotting. We can plot a true color image from the dataset by specifying the necessary bands.

In [ ]:
aspect_ratio_composite = s2_data_composite.x.shape[0] / s2_data_composite.y.shape[0]

In [ ]:
s2_data_composite[["red", "green", "blue"]].to_array("band").plot.imshow(rgb="band", robust=True, size=5, aspect=aspect_ratio_composite)

We can also plot any single band from the dataset like so:

In [ ]:
s2_data_composite[["nir"]].to_array("band").plot(robust=True)

Let's say we want to take a look at the time series for a single band. We can do this by taking the average in the spatial dimensions (x and y) for each timestamp and plotting the values.

In [ ]:
s2_data_mean_time_series = s2_data.mean(dim=["y", "x"])

Then we can extract the time series for one of bands. Near infrared is interesting because it is useful in determing seasonal phenology.

In [ ]:
nir_time_series = s2_data_mean_time_series["nir"]

We can plot a line plot for the near infrared averages for each timestamp.

In [ ]:
plt.figure(figsize=(10, 6))
nir_time_series.plot(label='Near Infrared Band', marker='o')

plt.title("Time Series of Mean Near Infrared Band Values")
plt.xlabel("Time")
plt.ylabel("Mean Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Dealing with cloud-covered and invalid pixels

To mask out clouds and invalid pixels from Sentinel-2 data, we can use the 'SCL' (Scene Classification Layer) band, which includes cloud and no data information. The SCL band classifies each pixel in the Sentinel-2 image, including cloud-related classes such as cloud shadows, medium-probability clouds, high-probability clouds, and cirrus clouds. By masking out these classes as well as those with the no data value, we can ignore unwanted pixels in the dataset.

First we need to declare the cloud-related SCL classes. The common classes for clouds are:

> Cloud Shadows: 3
>
> Low probability clouds: 7
>
> Medium probability clouds: 8
> 
> High probability clouds: 9
> 
> Thin cirrus: 10

In order to mask out these cloud classes, we'll create a boolean mask using `xarray`'s `.isin()` method. We will use these masks to filter out cloud-covered and invalid pixels in the red, green, blue, and NIR bands.

In [ ]:
cloud_classes = [3, 7, 8, 9, 10]  # Cloud-related SCL classes
cloud_mask = s2_data['scl'].isin(cloud_classes)
masked_s2_data = s2_data[['red', 'green', 'blue', 'nir']].where(~cloud_mask, drop=False)  # Keep all pixels

We will do the same for the no data "class."

In [ ]:
nodata_value =  s2_data['scl'].nodata
nodata_mask = s2_data['scl'].isin(nodata_value)
masked_s2_data = masked_s2_data[['red', 'green', 'blue', 'nir']].where(~nodata_value, drop=False)

In [ ]:
masked_s2_data

Let's examine the masked data to verify the results. To do this, let's composite the imagery again to get a temporal average.

In [ ]:
masked_s2_data_composite = masked_s2_data.mean(dim='time')

In [ ]:
masked_s2_data_composite

Now we can plot an image.

In [ ]:
masked_s2_data_composite[['red', 'green', 'blue']].to_array("band").plot.imshow(rgb="band", robust=True, size=5, aspect=aspect_ratio_composite)

The image plot is looking a bit over-exposed. Let's adjust the value ranges to better fit the raster data.

In [ ]:
masked_s2_data_composite[['red', 'green', 'blue']].to_array("band").plot.imshow(rgb="band", robust=True, size=5, aspect=aspect_ratio_composite, vmin=0, vmax=2000)

That looks pretty good! Now we have a cloud-masked composite. We can use this to get a clearer signal of near infrared over time.

## Calculate a spectral index

In this next section, we will use our masked multi-spectral data to calculate a relevant spectral index for tasks such as crop type classification.

**NDVI (Normalized Difference Vegetation Index)** is a widely used index to assess vegetation health and density. It quantifies the difference between near-infrared (NIR) light, which vegetation strongly reflects, and red light, which vegetation absorbs. Higher NDVI values typically indicate healthier or denser vegetation, while lower or negative values suggest sparse or no vegetation (e.g., bare soil, urban areas, or water).

In [ ]:
masked_s2_ndvi = (masked_s2_data["nir"] - masked_s2_data["red"]) / (masked_s2_data["nir"] + masked_s2_data["red"])

Now let's make a time series again and plot it.

In [ ]:
masked_s2_ndvi_mean_time_series = masked_s2_ndvi.mean(dim=["y", "x"])
plt.figure(figsize=(10, 6))
masked_s2_ndvi_mean_time_series.plot(label='NDVI', marker='o')

plt.title("Time Series of Mean NDVI Values")
plt.xlabel("Time")
plt.ylabel("Mean Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

The dataset titled **"10,000 Crop Field Boundaries across India"** (by Wang et al., 2022) contains manually delineated boundaries of 10,000 crop fields across India, created through the analysis of high-resolution satellite imagery from Airbus SPOT. The dataset is specifically designed to support research and applications in crop field mapping and agricultural monitoring, particularly in smallholder farming systems.

This dataset aims to advance methods for identifying and analyzing crop fields at scale, enabling better understanding and management of smallholder farming landscapes. For further details and access to the dataset and model weights, visit the [Zenodo record](https://zenodo.org/record/7315090).

We will use this dataset to demonstrate scalably vector dataset visualization with Lonboard and then subsquently compute zonal statistics using the polygons, our subdistrict and a NDVI raster.

Let's start by reading in the shapefile.

In [ ]:
field_boundaries_india = gpd.read_file("./7315090/india_10k_fields.shp")

In [ ]:
field_boundaries_india

And then visualize it:

In [ ]:
layer = PolygonLayer.from_geopandas(
    field_boundaries_india,
    get_line_width=20,  # width in default units (meters)
    line_width_min_pixels=0.2,  # minimum width when zoomed out
    get_fill_color=[255, 0, 0],  # red
    get_line_color=[37, 36, 34],  # dark border color
)

In [ ]:
m = Map(layer)
m

Now let's get all of the field boundaries within the subdistrict we've been focusing on.

In [ ]:
clipped_field_boundaries_india = field_boundaries_india.to_crs(epsg=4326).clip(scene_geom)

In [ ]:
len(clipped_field_boundaries_india)

Let's visualize the clipped results:

In [ ]:
layer = PolygonLayer.from_geopandas(
    clipped_field_boundaries_india,
    get_line_width=20,  # width in default units (meters)
    line_width_min_pixels=0.2,  # minimum width when zoomed out
    get_fill_color=[255, 0, 0],  # red
    get_line_color=[37, 36, 34],  # dark border color
)

m = Map(layer)
m

Now, let's just take one field boundary to compute zonal statistics for:

In [ ]:
clipped_field_boundaries_india_single = clipped_field_boundaries_india.head(1)

In [ ]:
viz(clipped_field_boundaries_india_single)

Get the raster data for the bounds of this polygon:

In [ ]:
s2_data = odc.stac.load(
    items=s2_items,
    bands=["red", "green", "blue", "nir", "scl"],
    bbox=clipped_field_boundaries_india_single.total_bounds,
    progress=tqdm,
    epsg=4326,
)

Get the cloud and no data masks.

In [ ]:
cloud_mask = s2_data['scl'].isin(cloud_classes)
nodata_value =  s2_data['scl'].nodata
nodata_mask = s2_data['scl'].isin(nodata_value)

Mask the non-SCL bands.

In [ ]:
nodata_masked_s2_data = s2_data[['red', 'green', 'blue', 'nir']].where(~nodata_value, drop=False)  # Keep all pixels
cloud_masked_s2_data = nodata_masked_s2_data[['red', 'green', 'blue', 'nir']].where(~cloud_mask, drop=False)  # Keep all pixels
masked_s2_data = cloud_masked_s2_data.copy()

Calculate NDVI for this new Sentinel-2 raster.

In [ ]:
masked_s2_ndvi = (masked_s2_data["nir"] - masked_s2_data["red"]) / (masked_s2_data["nir"] + masked_s2_data["red"])

masked_s2_ndvi_composite = masked_s2_ndvi.mean(dim=["time"])

In [ ]:
masked_s2_data

Now let's calculate zome basic statistics for NDVI within this field boundary polgon.

In [ ]:
# Calculate zonal stats
stats = zonal_stats(
    clipped_field_boundaries_india_single.to_crs(epsg=32643), #clipped_field_boundaries_india.to_crs(epsg=32643),
    masked_s2_ndvi_composite.values,
    affine=masked_s2_ndvi_composite.rio.transform(),
    stats=["mean", "min", "max", "sum"],
    geojson_out=True
)

# Convert the results back to a GeoDataFrame
zonal_stats_gdf = gpd.GeoDataFrame.from_features(stats)

In [ ]:
zonal_stats_gdf